# FitFood sales analysis and predictions

**Semester Project for Decision Systems 2020/2021 Course**

The goal of the competition is to create an efficient model for predicting whether the total 14-days sales of a particular product, offered by the Fitfood company at one of their FitBoxy locations in Poland, will exceed four pieces

## Task description

Provided data describe a short-term sales history of products at various point of sales (PoS). The target attribute will_it_sell tells if in the following 14 day period the total sales of a given product at a particular location will be at least 4 pcs. 

The data is very similar to the one from the second graded task, with the difference that the sets in this challenge do not contain any random probes (which were deliberately added to the data from the second graded task for evaluation purposes). 

The data tables are provided as two CSV files with the ';' separator sign. They can be downloaded after the registration for the challenge. Both files (training and test sets) have exactly the same format but all the values from the will_it_sell column in the test set are missing.

The evaluation metric will be AUC. During the challenge, your solutions will be evaluated on a small fraction of the test set, and your best preliminary AUC score will be displayed on the public Leaderboard. 

The submission format: the solutions need to be submitted as text files with predictions. The file should have exactly the same number of rows as the test data table. In each row, it should contain exactly one real number expressing the likeliness that the correct target value for the corresponding test set instance is 1.

## Solution

#### Author of the solution: Krzysztof Piesiewicz

#### Contents of the notebook:
  - [Loading the data](#Loading-the-data)
  - [Preliminary analysis and preprocessing](#Preliminary-analysis-and-preprocessing)
    - [Size attribute](#Size-attribute)
    - [Storage temperature attribute](#Storage-temperature-attribute)
    - [Generalizing some discrete attributes](#Generalizing-some-discrete-attributes)
    - [Preprocessing the data](#Preprocessing-the-data)
  - [Spliting data for training and validation](#Spliting-data-for-training-and-validation)
  - [Chalange test set preparation and answer saver](#Chalange-test-set-preparation-and-answer-saver)
  - [Features selection](#Features-selection)
    - [Analysis of correlations with target value](#Analysis-of-correlations-with-target-value)
    - [Features importance with random forrest](#Features-importance-with-random-forrest)
  - [Training on all the features](#Training-on-all-the-features)
    - [More samples in a leaf](#More-samples-in-a-leaf)
    - [Less samples in a leaf](#Less-samples-in-a-leaf)
    - [Training each estimator on more samples](#Training-each-estimator-on-more-samples)
  - [Fast comparison of feature selections with the same classifiers](#Fast-comparison-of-feature-selections-with-the-same-classifiers)
    - [The most important features with no information about locations](#The-most-important-features-with-no-information-about-locations)
    - [The most correlated with target value](#The-most-correlated-with-target-value)
  - [Training on the 85 most important features](#Training-on-the-85-most-important-features)
  - [Let's consider time dependecies problem](#Let's-consider-time-dependecies-problem)
    - [Benchmark of ExtraTreesClassifiers with season features and without](#Benchmark-of-ExtraTreesClassifiers-with-season-features-and-without)
  - [ExtraTreesClassifier and GradientBoosting with no season features trained on all data](#ExtraTreesClassifier-and-GradientBoosting-with-no-season-features-trained-on-all-data)
    - [ExtraTreesClassifier](#ExtraTreesClassifier)
    - [GradientBoosting](#GradientBoosting)
  - [Combining multiple answers into a final one](#Combining-multiple-answers-into-a-final-one)
  - [Summary - lessons learned](#Summary-lessons-learned)

## Loading the data

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 150)

In [3]:
train_data = pd.read_csv("FitFood_competition_data_training.csv", sep=";")

## Preliminary analysis and preprocessing

In [4]:
df = train_data
attrs = set(df.keys())
df.describe(include='all')

,will_it_sell,pos_id,product_id_unified,company_id,category_id,category_name,product_name,partner_product,address_city,diet,size,cooking_time,cooking_mv,cooking_ov,storage_temp,vat,bialko_100,weglow_100,cukry_calk,tluszcz_nasyc_calk,energia_calk,energia_100,tluszcz_nasyc_100,blonnik_100,tluszcz_calk,bialko_calk,sol_100,weglow_calk,blonnik_calk,tluszcz_100,cukry_100,sol_calk,weekday,quarter,month,week,qty_lag1,qty_lag2,qty_lag3,qty_lag4,qty_lag5,qty_lag6,qty_lag7,qty_lag8,qty_lag9,qty_lag10,qty_lag11,qty_lag12,qty_lag13,qty_lag14,meanLastPeriod_lag1,meanLastPeriod_lag2,meanLastPeriod_lag3,meanLastPeriod_lag4,meanLastPeriod_lag5,meanLastPeriod_lag6,meanLastPeriod_lag7,meanLastPeriod_lag1_lag7_diff,sdLastPeriod_lag1,sdLastPeriod_lag2,sdLastPeriod_lag3,sdLastPeriod_lag4,sdLastPeriod_lag5,sdLastPeriod_lag6,sdLastPeriod_lag7,minLastPeriod_lag1,minLastPeriod_lag7,minLastPeriod_lag1_lag7_diff,maxLastPeriod_lag1,maxLastPeriod_lag7,maxLastPeriod_lag1_lag7_diff,diff1_lag1,diff1_lag7,diff1_lag1_lag7_diff,diffLagPeriod_lag1,diffLagPeriod_lag7,diffLagPeriod_lag1_lag7_diff,mean_diff1_lag1,mean_diff1_lag7,mean_diff1_lag1_lag7_diff,sum_qty,avg_discount_mean_value_lag1,avg_discount_count_lag1,avg_from_blik_lag1,avg_from_paypass_lag1,avg_from_payu_lag1,avg_total_lag1,avg_total_to_discount_lag1,avg_total_base_lag1,avg_sum_fv_lag1,avg_transaction_discount_count_lag1,roc1_lag1,rocPeriod_lag1,days_since_prev_delivery,sales_since_prev_delivery,available_products,is_delivery_day
count,5.360496e+06,5360496,5.360496e+06,5360496,5360496,5360496,5360496,5.360496e+06,5360496,2840752,5360496,3632502,2336346.0,2336346.0,3930214,5.360496e+06,4.999443e+06,4.999443e+06,4.998511e+06,4.998511e+06,4.999443e+06,4.999443e+06,4.998511e+06,4.070153e+06,4.999443e+06,4.999443e+06,4.998511e+06,4.999443e+06,4.070153e+06,4.999443e+06,4.998511e+06,4.998511e+06,5360496,5360496,5.360496e+06,5.360496e+06,5.331152e+06,5.301896e+06,5.272670e+06,5.243488e+06,5.214306e+06,5.185124e+06,5.155944e+06,5.126791e+06,5.097737e+06,5.068781e+06,5.039988e+06,5.011318e+06,4.982648e+06,4.953978e+06,5.155944e+06,5.126791e+06,5.097737e+06,5.068781e+06,5.039988e+06,5.011318e+06,4.982648e+06,4.982648e+06,5.155944e+06,5.126791e+06,5.097737e+06,5.068781e+06,5.039988e+06,5.011318e+06,4.982648e+06,5.155944e+06,4.982648e+06,4.982648e+06,5.155944e+06,4.982648e+06,4.982648e+06,5.301896e+06,5.126791e+06,5.126791e+06,5.126791e+06,4.953978e+06,4.953978e+06,5.126791e+06,4.953978e+06,4.953978e+06,5.185124e+06,5.331152e+06,5.331152e+06,5.331152e+06,5.331152e+06,5.331152e+06,5.331152e+06,5.331152e+06,5.331152e+06,5331152.0,5.331152e+06,5.331152e+06,5.331152e+06,4.779475e+06,4.779475e+06,4.805004e+06,5.360496e+06
unique,NaN,375,NaN,298,14,14,137,NaN,33,10,22,1,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,59fc5325c94b722506678bd1,NaN,5a587706cf5c8134b3a9891d,5a6f110ca0899f5ca2f7d6e9,Dania Lunch Duże,Dyniowe curry z indykiem,NaN,Warszawa,Dieta Samuraja,350g,2-3 min.,NaN,NaN,2-5 °C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,czwartek,Q3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,25498,NaN,101533,1144201,1144201,112303,NaN,2353704,994953,1195365,3632502,NaN,NaN,3477491,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,776919,2034382,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [5]:
df.sample(n=50)

,will_it_sell,pos_id,product_id_unified,company_id,category_id,category_name,product_name,partner_product,address_city,diet,size,cooking_time,cooking_mv,cooking_ov,storage_temp,vat,bialko_100,weglow_100,cukry_calk,tluszcz_nasyc_calk,energia_calk,energia_100,tluszcz_nasyc_100,blonnik_100,tluszcz_calk,bialko_calk,sol_100,weglow_calk,blonnik_calk,tluszcz_100,cukry_100,sol_calk,weekday,quarter,month,week,qty_lag1,qty_lag2,qty_lag3,qty_lag4,qty_lag5,qty_lag6,qty_lag7,qty_lag8,qty_lag9,qty_lag10,qty_lag11,qty_lag12,qty_lag13,qty_lag14,meanLastPeriod_lag1,meanLastPeriod_lag2,meanLastPeriod_lag3,meanLastPeriod_lag4,meanLastPeriod_lag5,meanLastPeriod_lag6,meanLastPeriod_lag7,meanLastPeriod_lag1_lag7_diff,sdLastPeriod_lag1,sdLastPeriod_lag2,sdLastPeriod_lag3,sdLastPeriod_lag4,sdLastPeriod_lag5,sdLastPeriod_lag6,sdLastPeriod_lag7,minLastPeriod_lag1,minLastPeriod_lag7,minLastPeriod_lag1_lag7_diff,maxLastPeriod_lag1,maxLastPeriod_lag7,maxLastPeriod_lag1_lag7_diff,diff1_lag1,diff1_lag7,diff1_lag1_lag7_diff,diffLagPeriod_lag1,diffLagPeriod_lag7,diffLagPeriod_lag1_lag7_diff,mean_diff1_lag1,mean_diff1_lag7,mean_diff1_lag1_lag7_diff,sum_qty,avg_discount_mean_value_lag1,avg_discount_count_lag1,avg_from_blik_lag1,avg_from_paypass_lag1,avg_from_payu_lag1,avg_total_lag1,avg_total_to_discount_lag1,avg_total_base_lag1,avg_sum_fv_lag1,avg_transaction_discount_count_lag1,roc1_lag1,rocPeriod_lag1,days_since_prev_delivery,sales_since_prev_delivery,available_products,is_delivery_day
4885010,0,5b4893ddee2a423f39dacbf6,1156,5b39bb530663ab48e336173e,591301c83dd75608a9c2ef1b,Napoje,Smoothie BeRAW Breakfast energy,0,Skawina,NaN,250ml,NaN,NaN,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sobota,Q4,10,43,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,2.0,0.0,0.0,0
2160445,0,5cab37f2a77669455cbbaa78,1067,5caaeaa8822b5e2d312ac807,5cd1a4d32b10792bc08dab31,Pan Pomidor - Pierogi,"Pierogi z dynią, jarmużem, quinoą i kolendrą",0,Wrocław,NaN,240g,2-3 min.,NaN,NaN,1-6 °C,5,3.90,23.0,1.1,0.40,155.0,155.0,0.4,2.3,4.80,3.9,1.10,23.0,2.3,4.8,1.1,1.100,poniedziałek,Q3,9,37,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,31.0,0.0,0.0,0
514033,0,5a37f93f753e9f3591458dd7,1189,5af3e4736e089c600a14dd86,590053bdc5c79d3575eb44f6,Zupy,Barszcz z mlekiem kokosowym,0,Warszawa,Zupy,300g,2-3 min.,1.0,1.0,2-5 °C,5,1.20,6.6,12.6,4.10,130.3,43.4,1.4,1.9,4.70,3.5,0.40,199.0,5.6,1.6,4.2,1.300,wtorek,Q3,9,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,7.0,0.0,1.0,1
508094,0,5b2b8fa60663ab48e334dd2b,1063,5b30ea120663ab48e3354bfc,59005cd6c5c79d3575eb450d,Przekąski,BeRAW Baton protein 38% - surowe kakao w gorzk...,0,Kraków,NaN,60g,NaN,NaN,NaN,NaN,23,38.00,36.0,13.2,3.24,229.8,383.0,5.4,NaN,5.16,22.8,0.06,21.6,NaN,8.6,22.0,0.036,środa,Q2,6,26,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.285714,0.285714,0.285714,0.285714,0.285714,0.000000,0.000000,0.285714,0.755929,0.755929,0.755929,0.755929,0.755929,0.000000,0.000000,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,2.0,0.0,0.0,0.000,0.000000,1.0000

The fist glance at the table above leads to decuding that cooking time in oven or microwave oven are not varrying variables.

In [6]:
all(df.cooking_mv.notna() == df.cooking_ov.notna())

True

In [7]:
inds = df[df.cooking_mv.notna()].index
all(df.cooking_mv[inds] == df.cooking_ov[inds])

True

Atrributes cooking_mv and cooking_ov are not distinguishable

In [8]:
def impute_alt_or_zero(x, y):
    if not np.isnan(x) and not np.isnan(y):
        return max(x, y)
    if not np.isnan(x):
        return x
    if not np.isnan(y):
        return y
    return .0

def join_cooking_mv_and_ov(df):
    df["cooking_mv_or_ov"] = np.vectorize(impute_alt_or_zero)(df.cooking_mv, df.cooking_ov)

In [9]:
attrs_to_be_removed = {'cooking_mv', 'cooking_ov'}
numeric_attrs = set(df.select_dtypes(include=np.number).columns.to_list())

In [10]:
attrs_with_ids = set(filter(lambda c: "id" in c, attrs))
attrs_with_ids

{'category_id', 'company_id', 'pos_id', 'product_id_unified'}

In [11]:
attrs_with_names = set(filter(lambda c: "name" in c, attrs))
attrs_with_names

{'category_name', 'product_name'}

In [12]:
numeric_attrs -= attrs_with_ids
discrete_attrs = attrs - numeric_attrs
discrete_attrs

{'address_city',
 'category_id',
 'category_name',
 'company_id',
 'cooking_time',
 'diet',
 'pos_id',
 'product_id_unified',
 'product_name',
 'quarter',
 'size',
 'storage_temp',
 'weekday'}

In [13]:
discrete_descr = df[discrete_attrs].describe(include="all")
discrete_descr

,company_id,product_id_unified,diet,category_name,address_city,product_name,weekday,size,storage_temp,cooking_time,category_id,pos_id,quarter
count,5360496,5.360496e+06,2840752,5360496,5360496,5360496,5360496,5360496,3930214,3632502,5360496,5360496,5360496
unique,298,NaN,10,14,33,137,7,22,5,1,14,375,4
top,5a587706cf5c8134b3a9891d,NaN,Dieta Samuraja,Dania Lunch Duże,Warszawa,Dyniowe curry z indykiem,czwartek,350g,2-5 °C,2-3 min.,5a6f110ca0899f5ca2f7d6e9,59fc5325c94b722506678bd1,Q3
freq,101533,NaN,994953,1144201,2353704,112303,776919,1195365,3477491,3632502,1144201,25498,2034382
mean,NaN,1.104991e+03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,5.583013e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,1.004000e+03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,1.057000e+03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,1.117000e+03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,1.152000e+03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


I remove category and product ids in favour of the names.

In [14]:
attrs_to_be_removed.update({'category_id', 'product_id_unified'})
discrete_attrs = attrs - numeric_attrs - attrs_to_be_removed

### Size attribute

In [15]:
list(pd.unique(df["size"]))

['35g',
 '350g',
 '240g',
 '500g',
 '250ml',
 '180g',
 '50g',
 '300g',
 '230g',
 '400g',
 '40g',
 '60g',
 '250g',
 '370g',
 '380g',
 '300ml',
 '200ml',
 '120g',
 '85g',
 '220g',
 '2x68g',
 '100g']

Size attribute could be convert to numeric weight and volume attributes.

In [16]:
import re

def multiply_all_numbers_from_str(str):
    return np.prod(list(map(float, re.findall(r"\d+", str))))

def get_weight(size):
    if not size.endswith("g"):
        return np.nan
    return multiply_all_numbers_from_str(size)

def get_volume(size):
    if not size.endswith("ml"):
        return np.nan
    return multiply_all_numbers_from_str(size)

def convert_size_to_weight_and_volume(df):
    df["weight"] = np.vectorize(get_weight)(df["size"])
    df["volume"] = np.vectorize(get_volume)(df["size"])

In [17]:
attrs_to_be_removed.add("size")
discrete_attrs.remove("size")
numeric_attrs.update({"weight", "volume"})

### Storage temperature attribute

In [18]:
pd.unique(df.storage_temp)

array([nan, '2-5 °C', '1-6 °C', '2-6 °C', '0-6 °C', '0-7 °C'],
      dtype=object)

I will change storage temperature to stored in fridge attribute.

In [19]:
def convert_storage_temp_to_in_fridge(df):
    df["stored_in_fridge"] = df["storage_temp"].notna() * 1.0

attrs_to_be_removed.add("storage_temp")
discrete_attrs.remove("storage_temp")
numeric_attrs.add("stored_in_fridge")

### Generalizing some discrete attributes

There are some discrete attributes with no `NA` in any record and some of them should be generalizable.

In [20]:
counts = discrete_descr.loc["count",]
counts[counts == df.shape[0]].keys().intersection(discrete_attrs)

Index(['company_id', 'category_name', 'address_city', 'product_name',
       'weekday', 'pos_id', 'quarter'],
      dtype='object')

'weekday' and 'quarter' attributes are not extensible so I will focus on 'pos_id', 'company_id', 'product_name', 'category_name', 'address_city'.

#### Sales locations

In [21]:
pos_counts = df.pos_id.value_counts()
pos_counts

59fc5325c94b722506678bd1    25498
595e36122e84531cd204096b    24850
5b61699404852c4704a26f28    24658
5b3381160663ab48e335895f    24620
5b1b963bcaef965005d0e6b0    24590
                            ...  
5dc694443bf8f15dc97ec3c7       78
5dc69e723bf8f15dc97ec4de       46
5dcd4c06dacdf109c179d79b       31
5dcd2fb58fc7d108e6b8bc09       29
5dcbccb4aa5f032f78d77579       28
Name: pos_id, Length: 375, dtype: int64

In [22]:
pos_counts[pos_counts < 300]

5dbff693bb783c5423163b8d    292
5dbac6883bf8f15dc97d4251    210
5dc28ac6b902d15de1f0152b    193
5c7901d27b6c863a5d521f21     93
5dc694443bf8f15dc97ec3c7     78
5dc69e723bf8f15dc97ec4de     46
5dcd4c06dacdf109c179d79b     31
5dcd2fb58fc7d108e6b8bc09     29
5dcbccb4aa5f032f78d77579     28
Name: pos_id, dtype: int64

I will delete information for locations with less than 100 sales records for training purposes. I expect the samples with the location information removed to help me generalize the predictive model for new data. I want it to predict sales in completely new locations.

In [23]:
pos_id_cats = pos_counts[pos_counts >= 100].index
pos_ids_to_be_forgotten = set(pos_counts.index.difference(pos_id_cats))
np.sum(pos_counts[pos_ids_to_be_forgotten])

305

#### Companies

In [24]:
company_counts = df.company_id.value_counts()
company_counts

5a587706cf5c8134b3a9891d    101533
5c48270855f3af637b69b13f     94045
5b9fa5e201f82e03b412bcff     81488
5b503c4a9586df16bbb6e07a     77930
5c924f69b6cb840dcac430fe     75667
                             ...  
5dc534a2fdcd0622bd7b34e5       239
5dbff3dd9f38f239c2ffeb0c       188
5e4550dd177446520bcd2c91       101
5d764e24bf7a586310594da4        72
5d764e2fd0c59c62f781ff36        46
Name: company_id, Length: 298, dtype: int64

In [25]:
company_counts[company_counts<300]

5dbff5229f38f239c2ffeb55    270
5ce2969fc828ba60d8093297    245
5dc534a2fdcd0622bd7b34e5    239
5dbff3dd9f38f239c2ffeb0c    188
5e4550dd177446520bcd2c91    101
5d764e24bf7a586310594da4     72
5d764e2fd0c59c62f781ff36     46
Name: company_id, dtype: int64

In [26]:
company_id_cats = company_counts[company_counts >= 200].index
companies_to_be_forgotten = company_counts.index.difference(company_id_cats)
np.sum(company_counts[companies_to_be_forgotten])

407

#### Products names

In [27]:
product_counts = df.product_name.value_counts()
product_counts

Dyniowe curry z indykiem                 112303
Ostra wołowina z kaszą gryczaną          109728
Wołowina z marchewkowym puree            108298
Risotto z indykiem i cukinią             107248
Szaszłyk z ryżem w kurkumie              104697
                                          ...  
Silny Łasuch - Łasuch i jego orzeszki       153
FitElixir - Black                           144
Silny Łasuch - 2w1                           95
All'arrabiata                                71
Pomidorowa z chilli                           9
Name: product_name, Length: 137, dtype: int64

In [28]:
product_counts[product_counts<400]

Silny Łasuch - Marchew w tropikach       342
Silny Łasuch - Cynamonowe jabłuszko      342
Chili con carne z ryżen                  230
ROŚLEKO - jaglano - orzechowe            218
Allarrabbiata                            203
Superfood - ZDROWIE - kakao - maca       193
Kaszotto z soczewicą i grzybami          190
ROŚLEKO - kakao                          184
Silny Łasuch - Łasuch i jego orzeszki    153
FitElixir - Black                        144
Silny Łasuch - 2w1                        95
All'arrabiata                             71
Pomidorowa z chilli                        9
Name: product_name, dtype: int64

I will delete names of products with less than 200 sales records for training purposes. I would like to predict sales for completely new products.

In [29]:
product_name_cats = product_counts[product_counts >= 200].index
products_names_to_be_forgotten = product_counts.index.difference(product_name_cats)
np.sum(product_counts[products_names_to_be_forgotten])

1039

#### Cattegories

In [30]:
category_counts = df.category_name.value_counts()
category_counts

Dania Lunch Duże         1144201
Przekąski                 929010
Dania Lunch Małe          797960
Napoje                    498033
Zupy                      486330
Makarony                  412650
Śniadania                 265078
Pan Pomidor - Zupy        258243
Sałatki                   208141
Pan Pomidor - Pierogi     166430
Mr Thai                   157172
Lucky Fish                 28050
Desery                      5557
DayUp                       3641
Name: category_name, dtype: int64

In [31]:
len(category_counts)

14

There only 14 product categories so I do nothing with them and I do not expect the presence of any new category.

#### Address cities

In [32]:
address_counts = df.address_city.value_counts()
address_counts

Warszawa                           2353704
Kraków                             1577282
Wrocław                             694811
Katowice                            234123
Skawina                              71726
11                                   50702
Gliwice                              41412
Niepołomice                          39114
Wieliczka                            38747
Kobierzyce, Bielany Wrocławskie      28283
Zabierzów                            25683
Balice                               22416
Wroclaw                              20324
Jagiellońska 74                      16850
30-001                               15393
Bielany Wrocławskie                  14924
Waszawa                              14923
Ruda Śląska                          14873
Prądnicka 65                         14257
Podłęże, Kraków                      11648
Wysoka, Wrocław                      11360
Bytom                                 8362
Wrocław, Biskupice Podgórne           7342
Wrocławiu  

I will delete address city information for the locations with less than 200 sales records. I would like to predict sales at unknown places.

In [33]:
address_city_cats = address_counts[address_counts >= 200].index
addresses_to_be_forgotten = address_counts.index.difference(address_city_cats)
np.sum(address_counts[addresses_to_be_forgotten])

386

#### Saving the indices of the samples with attributes values to be forgotten
I will use them for mantaining generality the model and validating the trained results.

In [34]:
missing_cats_samples_idxs = {}
missing_cats_samples_idxs["pos_id"] = np.where(df.pos_id.isin(pos_ids_to_be_forgotten))[0]
missing_cats_samples_idxs["company_id"] = np.where(df.company_id.isin(companies_to_be_forgotten))[0]
missing_cats_samples_idxs["product_name"] = np.where(df.product_name.isin(products_names_to_be_forgotten))[0]
missing_cats_samples_idxs["address_city"] = np.where(df.address_city.isin(addresses_to_be_forgotten))[0]

important_missings_attrs = ["pos_id", "company_id", "product_name", "address_city"]

### Preprocessing the data

In [35]:
cats_for_attr = {'pos_id': pos_id_cats,
                 'company_id': company_id_cats,
                 'product_name': product_name_cats,
                 'address_city': address_city_cats}

for attr in discrete_attrs - {'pos_id', 'company_id', 'product_name', 'address_city'}:
    _, uniques = pd.factorize(df[attr])
    cats_for_attr[attr] = uniques

In [36]:
def remove_unnecessary_attrs(df):
    for attr in attrs_to_be_removed:
        del df[attr]

def convert_to_categoricals(df):
    for attr, attr_cats in cats_for_attr.items():
        df[attr] = df[attr].astype(pd.CategoricalDtype(attr_cats))

def with_categoricals_as_dummies(df):
    return pd.get_dummies(df,dummy_na=True, sparse=True)

def preprocessed_data_in_place(df):
    convert_size_to_weight_and_volume(df)
    join_cooking_mv_and_ov(df)
    convert_storage_temp_to_in_fridge(df)
    convert_to_categoricals(df)
    remove_unnecessary_attrs(df)
    return with_categoricals_as_dummies(df)

## Spliting data for training and validation

### Set weights of samples and extract important ones

In [37]:
samples_weights = np.empty(df.shape[0])
samples_weights.fill(0.01)
important_indices = set()

for attr in important_missings_attrs:
    for idx in missing_cats_samples_idxs[attr]:
        samples_weights[idx] += 0.24
    important_indices.update(missing_cats_samples_idxs[attr])

important_indices = pd.Index(important_indices).unique()
    
important_samples = df.loc[important_indices, ]

avg_samples = df.drop(important_indices, axis=0)

### Create training and validation sets

In [38]:
from sklearn.model_selection import train_test_split
avg_samples_train, avg_samples_val = train_test_split(avg_samples, random_state=0, test_size=0.05)
impt_samples_train, impt_samples_val = train_test_split(important_samples, random_state=0, test_size=0.3)

In [39]:
samples_train = pd.concat([avg_samples_train, impt_samples_train]).sample(frac=1)
samples_val = pd.concat([avg_samples_val, impt_samples_val]).sample(frac=1)

In [40]:
def x_y_split(df):
    y = df.will_it_sell
    X = df.drop(["will_it_sell"], axis=1)
    return X, y
    
def as_sparse(df):
    return df.astype(pd.SparseDtype("float", np.nan))

In [41]:
X_train, y_train = x_y_split(samples_train)
X_val, y_val = x_y_split(samples_val)

X_train = as_sparse(preprocessed_data_in_place(X_train))
y_train = as_sparse(y_train)
X_val = as_sparse(preprocessed_data_in_place(X_val))
y_val = as_sparse(y_val)

In [4]:
X_impt_val, y_impt_val = x_y_split(impt_samples_val)
X_impt_val = as_sparse(preprocessed_data_in_place(X_impt_val))
y_impt_val = as_sparse(y_impt_val)

## Chalange test set preparation and answer saver

In [19]:
X_test = pd.read_csv("FitFood_competition_data_test.csv", sep=";")
X_test = preprocessed_data_in_place(X_test).drop(["will_it_sell"], 1)
X_test = X_test.astype(pd.SparseDtype("float", np.nan))

In [20]:
import datetime

def save_ans(ans, ans_prefix_name="ans"):
    cur_dt = datetime.datetime.today()
    str_dt = cur_dt.strftime("%y-%m-%d_%H-%M-%S")
    file_name = f"{ans_prefix_name}_{str_dt}"
    with open(file_name, "w") as f:
        for p in ans:
            print(p, file=f)
    print(f"The answer saved as '{file_name}'")
            
            
def predict_ans(cls, X_test):
    return cls.predict_proba(X_test)[:,1]

def predict_and_save_ans(cls, ans_prefix_name="ans", X_test=X_test):
    ans = predict_ans(cls, X_test)
    save_ans(ans, ans_prefix_name)

## Features selection

### Analysis of correlations with target value

In [43]:
df = preprocessed_data_in_place(df)
corrs = df.corrwith(df.will_it_sell)
corrs.sort_values(inplace=True, ascending=False)

In [9]:
corrs[abs(corrs)>=0.04]

will_it_sell                                       1.000000
sum_qty                                            0.512098
meanLastPeriod_lag1                                0.482397
sdLastPeriod_lag1                                  0.479201
meanLastPeriod_lag2                                0.471712
sdLastPeriod_lag2                                  0.467968
maxLastPeriod_lag1                                 0.467142
meanLastPeriod_lag3                                0.461228
sdLastPeriod_lag3                                  0.456877
meanLastPeriod_lag4                                0.450535
sdLastPeriod_lag4                                  0.445714
meanLastPeriod_lag5                                0.439661
sdLastPeriod_lag5                                  0.434390
meanLastPeriod_lag6                                0.428485
sdLastPeriod_lag6                                  0.422924
meanLastPeriod_lag7                                0.417058
sdLastPeriod_lag7                       

In [10]:
len(corrs[abs(corrs)>=0.04])

98

In [46]:
len(corrs[abs(corrs)>=0.01])

438

In [47]:
len(corrs[abs(corrs)>=0.025])

171

### Features importance with random forrest

In [50]:
X_small = X_val
y_small = y_val

In [51]:
from sklearn.ensemble import ExtraTreesClassifier

forest = ExtraTreesClassifier(n_estimators=250, random_state=0)

forest.fit(X_small, y_small)
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
indices = np.argsort(importances)[::-1]

In [8]:
print("Feature ranking:")

ftrs = X_small.keys()
for i in range(len(ftrs)):
    print(f"{i + 1}. feature {indices[i]} ({ftrs[indices[i]]}) - importance {importances[indices[i]]}")

Feature ranking:
1. feature 64 (sum_qty) - importance 0.02857332555538195
2. feature 19 (week) - importance 0.026891786531448355
3. feature 68 (avg_from_paypass_lag1) - importance 0.021251139815816302
4. feature 18 (month) - importance 0.020849031102915774
5. feature 79 (available_products) - importance 0.01962649657615115
6. feature 77 (days_since_prev_delivery) - importance 0.019343703637926778
7. feature 34 (meanLastPeriod_lag1) - importance 0.01388388024562896
8. feature 42 (sdLastPeriod_lag1) - importance 0.012889859686469315
9. feature 36 (meanLastPeriod_lag3) - importance 0.012379493106474331
10. feature 35 (meanLastPeriod_lag2) - importance 0.012144795825265719
11. feature 74 (avg_transaction_discount_count_lag1) - importance 0.011964420664176374
12. feature 45 (sdLastPeriod_lag4) - importance 0.011362275913776547
13. feature 76 (rocPeriod_lag1) - importance 0.011260772429316009
14. feature 72 (avg_total_base_lag1) - importance 0.011078064406763889
15. feature 78 (sales_since_p

In [18]:
def auc_score(cls, X_val=X_val, y_val=y_val):
    y_val_pred = cls.predict_proba(X_val)[:,1]
    from sklearn.metrics import roc_auc_score
    return roc_auc_score(y_val, y_val_pred)

## Training on all the features

### More samples in a leaf

In [54]:
trees = ExtraTreesClassifier(n_estimators=100, random_state=0, bootstrap=True,
                             min_samples_leaf=100, max_samples=0.2, n_jobs=-1)
trees.fit(X_train, y_train)

ExtraTreesClassifier(bootstrap=True, max_samples=0.2, min_samples_leaf=100,
                     n_jobs=-1, random_state=0)

In [57]:
auc_score(trees)

0.9053929549124441

In [70]:
auc_score(trees, X_impt_val, y_impt_val)

0.8302667523702393

In [78]:
predict_and_save_ans(trees, "ans_trees")

### Less samples in a leaf

In [62]:
trees1 = ExtraTreesClassifier(n_estimators=100, random_state=1, bootstrap=True,
                             min_samples_leaf=30, max_samples=0.2, n_jobs=-1)
trees1.fit(X_train, y_train)

ExtraTreesClassifier(bootstrap=True, max_samples=0.2, min_samples_leaf=30,
                     n_jobs=-1, random_state=1)

In [63]:
auc_score(trees1)

0.9175393820524615

In [69]:
auc_score(trees1, X_impt_val, y_impt_val)

0.8448296641491242

In [79]:
predict_and_save_ans(trees1, "ans_trees1")

### Training each estimator on more samples

In [80]:
trees2 = ExtraTreesClassifier(n_estimators=100, random_state=1, bootstrap=True,
                             min_samples_leaf=30, max_samples=0.7, n_jobs=-1)
trees2.fit(X_train, y_train)

ExtraTreesClassifier(bootstrap=True, max_samples=0.7, min_samples_leaf=30,
                     n_jobs=-1, random_state=1)

In [81]:
auc_score(trees2)

0.9300189710101461

In [82]:
auc_score(trees2, X_impt_val, y_impt_val)

0.8582878033103005

In [87]:
predict_and_save_ans(trees2, "ans_trees2")

## Fast comparison of feature selections with the same classifiers

I add samples weights at fitting stage in order to focus on important samples (more general ones with unknown locations, products names, categories names or companies ids).

### The most important features with no information about locations
*with the exception of Warsaw and Krakow*

In [9]:
impt_ftrs85 = X_small.keys()[indices[:85]]
not_impt_ftrs85 = X_small.keys()[indices[85:]]

In [10]:
impt_ftrs85

Index(['sum_qty', 'week', 'avg_from_paypass_lag1', 'month',
       'available_products', 'days_since_prev_delivery', 'meanLastPeriod_lag1',
       'sdLastPeriod_lag1', 'meanLastPeriod_lag3', 'meanLastPeriod_lag2',
       'avg_transaction_discount_count_lag1', 'sdLastPeriod_lag4',
       'rocPeriod_lag1', 'avg_total_base_lag1', 'sales_since_prev_delivery',
       'maxLastPeriod_lag1', 'meanLastPeriod_lag6', 'meanLastPeriod_lag4',
       'sdLastPeriod_lag7', 'avg_total_lag1', 'avg_from_blik_lag1',
       'meanLastPeriod_lag7', 'sdLastPeriod_lag5', 'sdLastPeriod_lag2',
       'meanLastPeriod_lag5', 'sdLastPeriod_lag3', 'sdLastPeriod_lag6',
       'product_name_Dyniowe curry z indykiem', 'maxLastPeriod_lag7',
       'qty_lag1', 'meanLastPeriod_lag1_lag7_diff', 'qty_lag3', 'qty_lag2',
       'quarter_Q3', 'diff1_lag1', 'qty_lag4', 'weekday_środa',
       'weekday_piątek', 'sol_calk', 'is_delivery_day', 'avg_from_payu_lag1',
       'weekday_czwartek', 'avg_total_to_discount_lag1', 'weekday_n

In [7]:
indices[:85]

array([ 64,  19,  68,  18,  79,  77,  34,  42,  36,  35,  74,  45,  76,
        72,  78,  52,  39,  37,  48,  70,  67,  40,  46,  43,  38,  44,
        47, 765,  53,  20,  41,  22,  21, 950,  55,  23, 941, 943,  17,
        80,  69, 940,  71, 945, 944, 942,  11, 946, 895, 951,  54,   2,
        16,   4,  24,   7,  57, 896,  60, 948,  75, 949, 761,   6,   3,
        25,  63,  12,  13,  81,  15,  30,  29,  32,  31,  26,  10,   8,
        84,   5,  14,   9,  28,  59,  33])

In [30]:
X_impt_ftrs85_train = X_train.drop(not_impt_ftrs85, axis=1)
X_impt_ftrs85_val = X_val.drop(not_impt_ftrs85, axis=1)
X_impt_ftrs85_impt_val = X_impt_val.drop(not_impt_ftrs85, axis=1)

In [9]:
import multiprocessing

multiprocessing.cpu_count()

40

In [43]:
trees_impt85_fast = ExtraTreesClassifier(n_estimators=20, random_state=1, bootstrap=True,
                                         max_features=30, min_samples_leaf=1e-4, max_samples=0.2, n_jobs=30,
                                         verbose=1)
trees_impt85_fast.fit(X_impt_ftrs85_train, y_train, samples_weights[X_impt_ftrs85_train.index,])

[Parallel(n_jobs=30)]: Using backend ThreadingBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done   3 out of  20 | elapsed: 24.0min remaining: 136.0min
[Parallel(n_jobs=30)]: Done  20 out of  20 | elapsed: 25.3min finished


ExtraTreesClassifier(bootstrap=True, max_features=30, max_samples=0.2,
                     min_samples_leaf=0.0001, n_estimators=20, n_jobs=30,
                     random_state=1, verbose=True)

In [44]:
auc_score(trees_impt85_fast, X_impt_ftrs85_val, y_val)

[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   2 out of  20 | elapsed:    0.8s remaining:    7.0s
[Parallel(n_jobs=20)]: Done  20 out of  20 | elapsed:    0.9s finished


0.9200937299058559

In [45]:
auc_score(trees_impt85_fast, X_impt_ftrs85_impt_val, y_impt_val)

[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   2 out of  20 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=20)]: Done  20 out of  20 | elapsed:    0.0s finished


0.8938916117628153

### The most correlated with target value
#### Top 97 features

In [9]:
ftrs_corr97 = corrs[abs(corrs)>=0.04].keys().drop("will_it_sell")
not_ftrs_corr97 = corrs[abs(corrs)<0.04].keys()

In [10]:
len(ftrs_corr97.intersection(impt_ftrs85))

63

In [11]:
len(ftrs_corr97.union(impt_ftrs85))

119

The most correlated ones and the most important ones are mostly the same features

In [26]:
X_ftrs_corr97_train = X_train.drop(not_ftrs_corr97, axis=1)
X_ftrs_corr97_val = X_val.drop(not_ftrs_corr97, axis=1)
X_ftrs_corr97_impt_val = X_impt_val.drop(not_ftrs_corr97, axis=1)

In [46]:
trees_corr97 = ExtraTreesClassifier(n_estimators=20, random_state=1, bootstrap=True,
                                    max_features=30, min_samples_leaf=1e-4, max_samples=0.2, n_jobs=30,
                                    verbose=1)
trees_corr97.fit(X_ftrs_corr97_train, y_train, samples_weights[X_ftrs_corr97_train.index,])

[Parallel(n_jobs=30)]: Using backend ThreadingBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done   3 out of  20 | elapsed: 17.3min remaining: 97.9min
[Parallel(n_jobs=30)]: Done  20 out of  20 | elapsed: 19.0min finished


ExtraTreesClassifier(bootstrap=True, max_features=30, max_samples=0.2,
                     min_samples_leaf=0.0001, n_estimators=20, n_jobs=30,
                     random_state=1, verbose=True)

In [47]:
auc_score(trees_corr97, X_ftrs_corr97_val, y_val)

[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   2 out of  20 | elapsed:    0.7s remaining:    6.2s
[Parallel(n_jobs=20)]: Done  20 out of  20 | elapsed:    0.8s finished


0.916871361045221

In [48]:
auc_score(trees_corr97, X_ftrs_corr97_impt_val, y_impt_val)

[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   2 out of  20 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=20)]: Done  20 out of  20 | elapsed:    0.0s finished


0.8846717820986661

#### Top 170 features

In [2]:
ftrs_corr170 = corrs[abs(corrs)>=0.025].keys().drop("will_it_sell")
not_ftrs_corr170 = corrs[abs(corrs)<0.025].keys()

In [3]:
len(ftrs_corr170)

170

In [4]:
X_ftrs_corr170_train = X_train.drop(not_ftrs_corr170, axis=1)
X_ftrs_corr170_val = X_val.drop(not_ftrs_corr170, axis=1)
X_ftrs_corr170_impt_val = X_impt_val.drop(not_ftrs_corr170, axis=1)

In [5]:
trees_corr170 = ExtraTreesClassifier(n_estimators=20, random_state=1, bootstrap=True,
                                    max_features=30, min_samples_leaf=1e-4, max_samples=0.2, n_jobs=30,
                                    verbose=1)
trees_corr170.fit(X_ftrs_corr170_train, y_train, samples_weights[X_ftrs_corr170_train.index,])

[Parallel(n_jobs=30)]: Using backend ThreadingBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done   3 out of  20 | elapsed: 11.3min remaining: 63.8min
[Parallel(n_jobs=30)]: Done  20 out of  20 | elapsed: 12.0min finished


ExtraTreesClassifier(bootstrap=True, max_features=30, max_samples=0.2,
                     min_samples_leaf=0.0001, n_estimators=20, n_jobs=30,
                     random_state=1, verbose=True)

In [6]:
auc_score(trees_corr170, X_ftrs_corr170_val, y_val)

[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   2 out of  20 | elapsed:    0.6s remaining:    5.3s
[Parallel(n_jobs=20)]: Done  20 out of  20 | elapsed:    0.8s finished


0.9144692038135699

In [7]:
auc_score(trees_corr170, X_ftrs_corr170_impt_val, y_impt_val)

[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   2 out of  20 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=20)]: Done  20 out of  20 | elapsed:    0.0s finished


0.8906275108468584

## Training on the 85 most important features
### Lots of estimators and high max_samples rate

In [31]:
trees_impt85 = ExtraTreesClassifier(n_estimators=100, random_state=1, bootstrap=True,
                                    max_features=30, min_samples_leaf=1e-4, max_samples=0.7, n_jobs=30,
                                    oob_score=True, verbose=1)
trees_impt85.fit(X_impt_ftrs85_train, y_train, samples_weights[X_impt_ftrs85_train.index,])

[Parallel(n_jobs=30)]: Using backend ThreadingBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done 100 out of 100 | elapsed: 447.6min finished


ExtraTreesClassifier(bootstrap=True, max_features=30, max_samples=0.7,
                     min_samples_leaf=0.0001, n_jobs=30, oob_score=True,
                     random_state=1, verbose=True)

In [32]:
auc_score(trees_impt85, X_impt_ftrs85_val, y_val)

[Parallel(n_jobs=30)]: Using backend ThreadingBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done 100 out of 100 | elapsed:    4.8s finished


0.9301058309562702

In [33]:
auc_score(trees_impt85, X_impt_ftrs85_impt_val, y_impt_val)

[Parallel(n_jobs=30)]: Using backend ThreadingBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done 100 out of 100 | elapsed:    0.0s finished


0.917754700305319

In [36]:
X_impt_ftrs85_test = X_test.drop(not_impt_ftrs85, axis=1)

In [37]:
predict_and_save_ans(trees_impt85, "ans_trees_impt85", X_impt_ftrs85_test)

[Parallel(n_jobs=30)]: Using backend ThreadingBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done 100 out of 100 | elapsed:   28.4s finished


### A few estimators with high max_samples rate but each feature is taken into account when splitting at every level

In [13]:
trees_impt85_best_splitting = ExtraTreesClassifier(n_estimators=10, random_state=1, bootstrap=True,
                                    max_features=85, min_samples_leaf=1e-4, max_samples=0.95, n_jobs=30,
                                    oob_score=True, verbose=1)
trees_impt85_best_splitting.fit(X_impt_ftrs85_train, y_train, samples_weights[X_impt_ftrs85_train.index,])

[Parallel(n_jobs=30)]: Using backend ThreadingBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done   6 out of  10 | elapsed: 164.4min remaining: 109.6min
[Parallel(n_jobs=30)]: Done  10 out of  10 | elapsed: 167.0min finished
/home/krzpiesiewicz/venv/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:541: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/home/krzpiesiewicz/venv/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:545: RuntimeWarning: invalid value encountered in true_divide
  decision = (predictions[k] /


ExtraTreesClassifier(bootstrap=True, max_features=85, max_samples=0.95,
                     min_samples_leaf=0.0001, n_estimators=10, n_jobs=30,
                     oob_score=True, random_state=1, verbose=True)

In [14]:
auc_score(trees_impt85_best_splitting, X_impt_ftrs85_val, y_val)

[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   2 out of  10 | elapsed:    0.4s remaining:    1.6s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:    0.5s finished


0.9374549951022586

In [15]:
auc_score(trees_impt85_best_splitting, X_impt_ftrs85_impt_val, y_impt_val)

[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   2 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:    0.0s finished


0.9264924473726499

In [17]:
predict_and_save_ans(trees_impt85_best_splitting, "ans_trees_impt85_best_splitting", X_impt_ftrs85_test)

[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   2 out of  10 | elapsed:    2.9s remaining:   11.5s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:    3.0s finished


### Let's fit an estimator for predicting sales at known locations
Its test predictions for known locations could be combined with predictions of more general estimator for unknown locations.

I add pos_ids features to the 85 most important ones.

In [25]:
impt_ftrs455 = X_small.keys()[indices[:455]]
not_impt_ftrs455 = X_small.keys()[indices[455:]]

In [26]:
impt_ftrs455

Index(['sum_qty', 'week', 'avg_from_paypass_lag1', 'month',
       'available_products', 'days_since_prev_delivery', 'meanLastPeriod_lag1',
       'sdLastPeriod_lag1', 'meanLastPeriod_lag3', 'meanLastPeriod_lag2',
       ...
       'company_id_5bbdc6a015c00937a0810fc7',
       'pos_id_5a44d792f9cdb15f7f7d9d49',
       'company_id_5b35c7a90663ab48e335be72',
       'pos_id_5b713f4fd8f80d6ebf31d49f', 'pos_id_5ba0d8dd01f82e03b41313a0',
       'company_id_5bd018b451478a069f6c34fa', 'product_name_Gnocchi primavera',
       'pos_id_5c94d70e1032d642b63487c2',
       'company_id_5c8f6c59c667da0ef079d9fc',
       'pos_id_5cc040375e70463ad9d520dd'],
      dtype='object', length=455)

In [27]:
X_impt_ftrs455_train = X_train.drop(not_impt_ftrs455, axis=1)
X_impt_ftrs455_val = X_val.drop(not_impt_ftrs455, axis=1)
X_impt_ftrs455_impt_val = X_impt_val.drop(not_impt_ftrs455, axis=1)

In [28]:
trees_impt455 = ExtraTreesClassifier(n_estimators=20, random_state=2, bootstrap=True,
                                     max_features=30, min_samples_leaf=1e-5, max_samples=0.7, n_jobs=30,
                                     verbose=1)
trees_impt455.fit(X_impt_ftrs455_train, y_train)

[Parallel(n_jobs=30)]: Using backend ThreadingBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done   3 out of  20 | elapsed: 57.7min remaining: 326.8min
[Parallel(n_jobs=30)]: Done  20 out of  20 | elapsed: 60.2min finished


ExtraTreesClassifier(bootstrap=True, max_features=30, max_samples=0.7,
                     min_samples_leaf=1e-05, n_estimators=20, n_jobs=30,
                     random_state=2, verbose=True)

In [29]:
auc_score(trees_impt455, X_impt_ftrs455_val, y_val)

[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   2 out of  20 | elapsed:    1.5s remaining:   13.9s
[Parallel(n_jobs=20)]: Done  20 out of  20 | elapsed:    1.6s finished


0.9314723596253695

In [30]:
auc_score(trees_impt455, X_impt_ftrs455_impt_val, y_impt_val)

[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   2 out of  20 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=20)]: Done  20 out of  20 | elapsed:    0.0s finished


0.8778924955809095

### Let's try with Gradient Boosting

In [6]:
from sklearn.ensemble import GradientBoostingClassifier
gb_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=8, min_samples_leaf=1e-4,
                                    subsample=0.01, validation_fraction=0.001,
                                    n_iter_no_change=3, random_state=3, verbose=3)
gb_clf.fit(X_impt_ftrs85_train, y_train)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           0.6617           0.0644           33.33m
         2           0.6287           0.0392           29.90m
         3           0.6041           0.0296           29.57m
         4           0.5772           0.0234           29.05m
         5           0.5567           0.0187           28.42m
         6           0.5374           0.0153           28.16m
         7           0.5212           0.0131           27.94m
         8           0.5193           0.0116           27.76m
         9           0.5130           0.0086           27.19m
        10           0.5037           0.0072           26.70m
        11           0.4985           0.0073           26.40m
        12           0.4888           0.0065           26.05m
        13           0.4850           0.0055           25.69m
        14           0.4833           0.0049           25.31m
        15           0.4709           0.0043           24.91m
       

GradientBoostingClassifier(max_depth=8, min_samples_leaf=0.0001,
                           n_iter_no_change=3, random_state=3, subsample=0.01,
                           validation_fraction=0.001, verbose=3)

In [7]:
auc_score(gb_clf, X_impt_ftrs85_val, y_val)

0.9233331500706501

In [8]:
auc_score(gb_clf, X_impt_ftrs85_impt_val, y_impt_val)

0.9132552627350152

In [5]:
predict_and_save_ans(gb_clf, "ans_gb_clf", X_impt_ftrs85_test)

## Let's consider time dependecies problem
Relating to features importance ranking, time features are at high positions. Note that test set samples are only from part of the year. I try to reach better generality by removing time/season information (with the except of weekdays).

In [26]:
np.sort(pd.unique(df.month))

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [25]:
np.sort(pd.unique(X_test.month).to_dense())

array([ 1.,  2., 11., 12.])

In [27]:
np.sort(pd.unique(df.week))

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52])

In [28]:
np.sort(pd.unique(X_test.week).to_dense())

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 48., 49., 50., 51.,
       52.])

In [15]:
time_ftrs = ['quarter_Q1', 'quarter_Q2', 'quarter_Q3', 'quarter_Q4', 'month', 'week']
X_impt_ftrs85_no_time_train = X_impt_ftrs85_train.drop(time_ftrs, axis=1)
X_impt_ftrs85_no_time_val = X_impt_ftrs85_val.drop(time_ftrs, axis=1)
X_impt_ftrs85_no_time_impt_val = X_impt_ftrs85_impt_val.drop(time_ftrs, axis=1)

In [16]:
X_impt_ftrs85_no_time_val.keys()

Index(['bialko_100', 'weglow_100', 'cukry_calk', 'tluszcz_nasyc_calk',
       'energia_calk', 'energia_100', 'tluszcz_nasyc_100', 'blonnik_100',
       'tluszcz_calk', 'bialko_calk', 'sol_100', 'weglow_calk', 'blonnik_calk',
       'tluszcz_100', 'cukry_100', 'sol_calk', 'qty_lag1', 'qty_lag2',
       'qty_lag3', 'qty_lag4', 'qty_lag5', 'qty_lag6', 'qty_lag7', 'qty_lag9',
       'qty_lag10', 'qty_lag11', 'qty_lag12', 'qty_lag13', 'qty_lag14',
       'meanLastPeriod_lag1', 'meanLastPeriod_lag2', 'meanLastPeriod_lag3',
       'meanLastPeriod_lag4', 'meanLastPeriod_lag5', 'meanLastPeriod_lag6',
       'meanLastPeriod_lag7', 'meanLastPeriod_lag1_lag7_diff',
       'sdLastPeriod_lag1', 'sdLastPeriod_lag2', 'sdLastPeriod_lag3',
       'sdLastPeriod_lag4', 'sdLastPeriod_lag5', 'sdLastPeriod_lag6',
       'sdLastPeriod_lag7', 'maxLastPeriod_lag1', 'maxLastPeriod_lag7',
       'maxLastPeriod_lag1_lag7_diff', 'diff1_lag1', 'diff1_lag1_lag7_diff',
       'diffLagPeriod_lag7', 'diffLagPeriod_lag1_

### Benchmark of ExtraTreesClassifiers with season features and without
#### Benchmark on my validations sets
I will test dropping time information with the same fast classifier which I used with `X_impt_ftrs85` for the first time when it scored `0.92009` and `0.89389` AUC on my validation sets.

In [17]:
trees_impt85_no_time_fast = ExtraTreesClassifier(n_estimators=20, random_state=1, bootstrap=True,
                                         max_features=30, min_samples_leaf=1e-4, max_samples=0.2, n_jobs=30,
                                         verbose=2)
trees_impt85_no_time_fast.fit(X_impt_ftrs85_no_time_train, y_train,
                              samples_weights[X_impt_ftrs85_no_time_train.index,])

[Parallel(n_jobs=30)]: Using backend ThreadingBackend with 30 concurrent workers.


building tree 1 of 20
building tree 2 of 20
building tree 3 of 20
building tree 4 of 20
building tree 5 of 20
building tree 6 of 20
building tree 7 of 20
building tree 8 of 20
building tree 9 of 20
building tree 10 of 20building tree 11 of 20
building tree 12 of 20
building tree 13 of 20

building tree 14 of 20
building tree 15 of 20
building tree 16 of 20
building tree 17 of 20
building tree 18 of 20
building tree 19 of 20
building tree 20 of 20


[Parallel(n_jobs=30)]: Done   5 out of  20 | elapsed: 23.4min remaining: 70.3min
[Parallel(n_jobs=30)]: Done  16 out of  20 | elapsed: 24.2min remaining:  6.1min
[Parallel(n_jobs=30)]: Done  20 out of  20 | elapsed: 24.3min finished


ExtraTreesClassifier(bootstrap=True, max_features=30, max_samples=0.2,
                     min_samples_leaf=0.0001, n_estimators=20, n_jobs=30,
                     random_state=1, verbose=2)

In [19]:
auc_score(trees_impt85_no_time_fast, X_impt_ftrs85_no_time_val, y_val)

[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   3 out of  20 | elapsed:    0.8s remaining:    4.3s
[Parallel(n_jobs=20)]: Done  14 out of  20 | elapsed:    0.8s remaining:    0.3s
[Parallel(n_jobs=20)]: Done  20 out of  20 | elapsed:    0.8s finished


0.9036358011163456

In [21]:
auc_score(trees_impt85_no_time_fast, X_impt_ftrs85_no_time_impt_val, y_impt_val)

[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   3 out of  20 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=20)]: Done  14 out of  20 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=20)]: Done  20 out of  20 | elapsed:    0.0s finished


0.8815985055439498

In [22]:
X_impt_ftrs85_no_time_test = X_impt_ftrs85_test.drop(time_ftrs, axis=1)

In [23]:
predict_and_save_ans(trees_impt85_no_time_fast, "trees_impt85_no_time_fast", X_impt_ftrs85_no_time_test)

[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   3 out of  20 | elapsed:    5.5s remaining:   31.3s
[Parallel(n_jobs=20)]: Done  14 out of  20 | elapsed:    5.8s remaining:    2.5s
[Parallel(n_jobs=20)]: Done  20 out of  20 | elapsed:    5.8s finished


These results are worse but both training and validation sets contain samples from each season. So the trees in previous model could be better fitted to specific time I should have split samples acca validation.
#### Benchmark on train and validation sets split by season

In [5]:
df85_impt_ftrs85 = df.drop(not_impt_ftrs85, axis=1)

samples_months_3to10_indices = np.where(df85_impt_ftrs85.month.isin(range(3, 11)))[0]

samples_impt_ftrs85_months_3to10 = df85_impt_ftrs85.loc[samples_months_3to10_indices, ]
samples_impt_ftrs85_months_11to2 = df85_impt_ftrs85.drop(samples_months_3to10_indices, axis=0)

X_impt_ftrs85_months_3to10, y_months_3to10 = x_y_split(samples_impt_ftrs85_months_3to10)
X_impt_ftrs85_months_11to2, y_months_11to2 = x_y_split(samples_impt_ftrs85_months_11to2)

In [6]:
X_impt_ftrs85_months_3to10 = as_sparse(X_impt_ftrs85_months_3to10)
y_months_3to10 = as_sparse(y_months_3to10)
X_impt_ftrs85_months_11to2 = as_sparse(X_impt_ftrs85_months_11to2)
y_months_11to2 = as_sparse(y_months_11to2)

In [7]:
time_ftrs = ['quarter_Q1', 'quarter_Q2', 'quarter_Q3', 'quarter_Q4', 'month', 'week']

In [8]:
X_impt_ftrs85_no_time_months_3to10 = X_impt_ftrs85_months_3to10.drop(time_ftrs, axis=1)
X_impt_ftrs85_no_time_months_11to2 = X_impt_ftrs85_months_11to2.drop(time_ftrs, axis=1)

In [9]:
trees_impt85_very_fast = ExtraTreesClassifier(n_estimators=10, random_state=1, bootstrap=True,
                                              max_features=30, min_samples_leaf=1e-4, max_samples=0.1,
                                              n_jobs=30, verbose=2)
trees_impt85_very_fast.fit(X_impt_ftrs85_months_3to10, y_months_3to10,
                           samples_weights[X_impt_ftrs85_months_3to10.index,])

[Parallel(n_jobs=30)]: Using backend ThreadingBackend with 30 concurrent workers.


building tree 1 of 10building tree 2 of 10

building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


[Parallel(n_jobs=30)]: Done   5 out of  10 | elapsed:  4.4min remaining:  4.4min
[Parallel(n_jobs=30)]: Done  10 out of  10 | elapsed:  4.4min finished


ExtraTreesClassifier(bootstrap=True, max_features=30, max_samples=0.1,
                     min_samples_leaf=0.0001, n_estimators=10, n_jobs=30,
                     random_state=1, verbose=2)

In [10]:
auc_score(trees_impt85_very_fast, X_impt_ftrs85_months_11to2, y_months_11to2)

[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   3 out of  10 | elapsed:    1.4s remaining:    3.4s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:    1.5s finished


0.8418574024096729

In [11]:
trees_impt85_no_time_very_fast = ExtraTreesClassifier(n_estimators=10, random_state=1, bootstrap=True,
                                                      max_features=30, min_samples_leaf=1e-4, max_samples=0.1,
                                                      n_jobs=30, verbose=2)
trees_impt85_no_time_very_fast.fit(X_impt_ftrs85_no_time_months_3to10, y_months_3to10,
                                   samples_weights[X_impt_ftrs85_no_time_months_3to10.index,])

[Parallel(n_jobs=30)]: Using backend ThreadingBackend with 30 concurrent workers.


building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


[Parallel(n_jobs=30)]: Done   5 out of  10 | elapsed:  4.3min remaining:  4.3min
[Parallel(n_jobs=30)]: Done  10 out of  10 | elapsed:  4.4min finished


ExtraTreesClassifier(bootstrap=True, max_features=30, max_samples=0.1,
                     min_samples_leaf=0.0001, n_estimators=10, n_jobs=30,
                     random_state=1, verbose=2)

In [12]:
auc_score(trees_impt85_no_time_very_fast, X_impt_ftrs85_no_time_months_11to2, y_months_11to2)

[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   3 out of  10 | elapsed:    1.4s remaining:    3.2s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:    1.4s finished


0.8473781705546937

Training with no season features outperformed the one with them
## ExtraTreesClassifier and GradientBoosting with no season features trained on all data
### ExtraTreesClassifier

In [3]:
not_impt_ftrs85_no_time = not_impt_ftrs85.append(pd.Index(time_ftrs))

In [4]:
X, y = x_y_split(df.drop(not_impt_ftrs85_no_time, axis=1))

In [6]:
X = as_sparse(X)
y = as_sparse(y)

In [23]:
trees_impt85_no_time_best_splitting = ExtraTreesClassifier(n_estimators=10, random_state=1, bootstrap=True,
                                                           max_features=79, min_samples_leaf=1e-4,
                                                           max_samples=0.95, n_jobs=40, verbose=2)
trees_impt85_no_time_best_splitting.fit(X, y, samples_weights[X.index,])

[Parallel(n_jobs=40)]: Using backend ThreadingBackend with 40 concurrent workers.


building tree 1 of 10building tree 2 of 10

building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


[Parallel(n_jobs=40)]: Done   3 out of  10 | elapsed: 99.2min remaining: 231.4min
[Parallel(n_jobs=40)]: Done  10 out of  10 | elapsed: 103.7min finished


ExtraTreesClassifier(bootstrap=True, max_features=79, max_samples=0.95,
                     min_samples_leaf=0.0001, n_estimators=10, n_jobs=40,
                     random_state=1, verbose=2)

In [7]:
X_test = X_test.drop(not_impt_ftrs85_no_time, axis=1)

In [26]:
predict_and_save_ans(trees_impt85_no_time_best_splitting, "ans_trees_impt85_no_time_best_splitting", X_test)

[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   3 out of  10 | elapsed:    3.8s remaining:    8.9s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:    3.9s finished


### GradientBoosting

In [9]:
from sklearn.ensemble import GradientBoostingClassifier
gb_clf_no_time = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=8,
                                            min_samples_leaf=1e-4,
                                            subsample=0.01, validation_fraction=0.001,
                                            n_iter_no_change=3, random_state=3, verbose=3)
gb_clf_no_time.fit(X, y)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           0.6674           0.0645           47.86m
         2           0.6276           0.0398           40.72m
         3           0.6057           0.0289           38.36m
         4           0.5750           0.0225           37.08m
         5           0.5641           0.0180           36.08m
         6           0.5410           0.0153           35.19m
         7           0.5314           0.0126           34.39m
         8           0.5122           0.0107           33.68m
         9           0.5075           0.0093           33.02m
        10           0.4979           0.0080           32.41m
        11           0.4934           0.0070           31.93m
        12           0.4910           0.0056           31.37m
        13           0.4863           0.0046           30.78m
        14           0.4795           0.0044           30.38m
        15           0.4741           0.0035           29.92m
       

GradientBoostingClassifier(max_depth=8, min_samples_leaf=0.0001,
                           n_iter_no_change=3, random_state=3, subsample=0.01,
                           validation_fraction=0.001, verbose=3)

In [10]:
predict_and_save_ans(gb_clf_no_time, "ans_gb_clf_no_time", X_test)

## Combining multiple answers into a final one

In [24]:
def combine_answers(answers, weights, masks):
    cb_ans = np.zeros(answers[0].shape)
    weights_sum = np.zeros(answers[0].shape)
    for ans, w, mask in zip(answers, weights, masks):
        cb_ans += ans * w * mask
        weights_sum += w * mask
    return cb_ans / weights_sum

In [20]:
def load_answer(file_name):
    ans = numpy.empty(X_test.shape[0])
    with open(file_name, "r") as f:
        for i, p in enumerate(f.readlines()):
            ans[i] = p

Finally I decided not to combine multiple answers...

## Summary - Lessons learned <a class="anchor" id="Summary-lessons-learned"></a>
To make a long story short I conclude that most important thing is creating not overfitted models that uses not too many features so that training samples could not be too finely split by irrelevant ones. Of course, there is also a need to set minimum number of samples required to be at a leaf node.

Due to the contest site evaluation my gradient boosting models outperforms my extra trees classifiers. My test does not confirm that but it is not trustworthy because my validation set is too similar to the training one. Another reason could be the fact I set max depth for gradient boosting trees.